# **Hacer solo un `runall` para evitar errores y revisar así!**

# 0. Utilidades

In [2]:
# Librerias
import pandas as pd
import re
from faker import Faker
from datetime import datetime

In [3]:
# Para inventar datos falsos:
fake = Faker()

In [4]:
# Carga de datos
books_df = pd.read_csv('GoodReads_Books.csv')
authors_df = pd.read_csv('GoodReads_Authors.csv')

In [5]:
books_df.head()

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,NaN,['Locus Award Nominee for Best Young Adult Boo...,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,06/21/03,['Bram Stoker Award for Works for Young Reader...,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,07/11/60,"['Pulitzer Prize for Fiction (1961)', 'Audie A...",4501075,"['2363896', '1333153', '573280', '149952', '80...",95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,9999999999999,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,01/28/13,[],2998241,"['1617567', '816659', '373311', '113934', '767...",94.0,"['United Kingdom', 'Derbyshire, England (Unite...",https://i.gr-assets.com/images/S/compressed.ph...,1983116,20452,NaN
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,10/05/05,"['Georgia Peach Book Award (2007)', 'Buxtehude...",4964519,"['1751460', '1113682', '1008686', '542017', '5...",78.0,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,1459448,14874,2.1


# 1. Busqueda de autores comunes (creación de llave foránea)

In [6]:
def extract_primary_author(author_string):
    """
    Esta función extrae el primer nombre de autor principal del campo 'author'.
    Ignora coautores y otros adicionales como parentesis, mayusculas, etc.
    """
    # Eliminar todo lo que esté entre paréntesis, incluyendo los paréntesis
    author_string = re.sub(r'\(.*?\)', '', author_string)
    # Tomar solo el primer fragmento antes de una coma (si la hay)
    primary_author = author_string.split(",")[0]
    # Limpiar espacios extra y convertir todo a minúsculas
    primary_author = ' '.join(primary_author.strip().lower().split())
    return primary_author

In [7]:
# Aplica la función a la columna 'author' en el dataset de libros
books_df['author_clean'] = books_df['author'].apply(extract_primary_author)

# Aplica la función a la columna 'name' en el dataset de autores
authors_df['name_clean'] = authors_df['name'].str.lower()

# Reordenar las columnas en books_df
cols_books = books_df.columns.tolist()
cols_books.insert(cols_books.index('author') + 1, cols_books.pop(cols_books.index('author_clean')))
books_df = books_df[cols_books]

# Reordenar las columnas en authors_df
cols_authors = authors_df.columns.tolist()
cols_authors.insert(cols_authors.index('name') + 1, cols_authors.pop(cols_authors.index('name_clean')))
authors_df = authors_df[cols_authors]

In [8]:
# Autores Unicos:
unique_authors = authors_df[authors_df['name_clean'].isin(books_df['author_clean'].unique())]
len(unique_authors)

17918

# 2. Creación de las 3 Tablas

## Libro

| **Nombre Original (Hito 1)**            | **Nombre Simplificado (para implementación)** | **Tipo de Atributo**     |
|------------------------------------------|-----------------------------------------------|---------------------------|
| BookId                                   | `bookid`                                      | `String`                  |
| Título                                   | `titulo`                                      | `String`                  |
| Autor                                    | `autor`                                       | `String`                  |
| Descripción                              | `descripcion`                                 | `String`                  |
| Idioma                                   | `idioma`                                      | `String`                  |
| Géneros                                  | `generos`                                     | `Array de Strings`        |
| Personajes                               | `personajes`                                  | `Array de Strings`        |
| Formato del Libro                        | `formato`                                     | `String`                  |
| Páginas                                  | `paginas`                                     | `Int`                     |
| Editor                                   | `editor`                                      | `String`                  |
| Fecha de publicación                     | `fecha_pub`                                   | `String`                  |
| Primera fecha de Publicación             | `fecha_pub_inicial`                           | `String`                  |
| Premios                                  | `premios`                                     | `String`                  |
| Escenario                             | `ambientacion`                                | `String`                  |


In [9]:
# Creamos DataFrame Libro:
Libro0 = books_df.copy()

# Eliminamos atributos que no necesitamos:
columns_to_drop = ['series', 'author', 'rating', 'isbn', 'numRatings', 'ratingsByStars', 'likedPercent', 'coverImg', 'bbeScore', 'bbeVotes', 'price', 'edition']
Libro0 = Libro0.drop(columns=columns_to_drop)

# Renombramos los atributos a utilizar:
Libro0.rename(columns={
        'bookId': 'bookid',
        'title': 'titulo',
        'author_clean': 'autor',
        'description': 'descripcion',
        'language' : 'idioma',
        'genres': 'generos',
        'characters': 'personajes',
        'bookFormat': 'formato',
        'pages': 'paginas',
        'publisher': 'editor',
        'publishDate': 'fecha_pub',
        'firstPublishDate': 'fecha_pub_inicial',
        'awards': 'premios',
        'setting': 'ambientacion'
    }, inplace=True)

# Resultado final:
Libro0.head()

,bookid,titulo,autor,descripcion,idioma,generos,personajes,formato,paginas,editor,fecha_pub,fecha_pub_inicial,premios,ambientacion
0,2767052-the-hunger-games,The Hunger Games,suzanne collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",Hardcover,374,Scholastic Press,09/14/08,NaN,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Pane..."
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,j.k. rowling,There is a door at the end of a silent corrido...,English,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",Paperback,870,Scholastic Inc.,09/28/04,06/21/03,['Bram Stoker Award for Works for Young Reader...,['Hogwarts School of Witchcraft and Wizardry (...
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,harper lee,The unforgettable novel of a childhood in a sl...,English,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",Paperback,324,Harper Perennial Modern Classics,05/23/06,07/11/60,"['Pulitzer Prize for Fiction (1961)', 'Audie A...","['Maycomb, Alabama (United States)']"
3,1885.Pride_and_Prejudice,Pride and Prejudice,jane austen,Alternate cover edition of ISBN 9780679783268S...,English,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",Paperback,279,Modern Library,10/10/00,01/28/13,[],"['United Kingdom', 'Derbyshire, England (Unite..."
4,41865.Twilight,Twilight,stephenie meyer,About three things I was absolutely positive.\...,English,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",Paperback,501,"Little, Brown and Company",09/06/06,10/05/05,"['Georgia Peach Book Award (2007)', 'Buxtehude...","['Forks, Washington (United States)', 'Phoenix..."


## Valoracion

| **Nombre Original (Hito 1)**                     | **Nombre Simplificado (para la implementación)** | **Tipo de Atributo**     |
|--------------------------------------------------|--------------------------------------------------|---------------------------|
| BookId                                           | `bookid`                                         | `String`                  |
| Calificación                                     | `calificacion`                                   | `Float`                   |
| No de Calificaciones                             | `num_calificaciones`                              | `Int`                     |
| Calificaciones por Estrellas                     | `estrellas`                                      | `Array de Strings`        |
| Aprobación                                       | `porcentaje_likes`                                     | `Int`                   |
| Puntuación BBE                                  | `puntuacion_bbe`                                 | `Float`                   |
| BBE Votos                                        | `votos_bbe`                                      | `Int`                     |



In [10]:
# Creamos DataFrame Valoracion:
Valoracion0 = books_df.copy()

# Eliminamos atributos que no necesitamos:
columns_to_keep = ['bookId', 'rating', 'numRatings', 'ratingsByStars', 'likedPercent', 'bbeScore', 'bbeVotes']
Valoracion0 = Valoracion0[columns_to_keep]

# Renombramos los atributos a utilizar:
Valoracion0.rename(columns={
        'bookId': 'bookid',
        'rating': 'calificacion',
        'numRatings': 'num_calificaciones',
        'ratingsByStars': 'estrellas',
        'likedPercent': 'porcentaje_likes',
        'bbeScore': 'puntuacion_bbe',
        'bbeVotes': 'votos_bbe'
    }, inplace=True)

# Resultado final:
Valoracion0

,bookid,calificacion,num_calificaciones,estrellas,porcentaje_likes,puntuacion_bbe,votos_bbe
0,2767052-the-hunger-games,4.33,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,2993816,30516
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,4.50,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,2632233,26923
2,2657.To_Kill_a_Mockingbird,4.28,4501075,"['2363896', '1333153', '573280', '149952', '80...",95.0,2269402,23328
3,1885.Pride_and_Prejudice,4.26,2998241,"['1617567', '816659', '373311', '113934', '767...",94.0,1983116,20452
4,41865.Twilight,3.60,4964519,"['1751460', '1113682', '1008686', '542017', '5...",78.0,1459448,14874
...,...,...,...,...,...,...,...
52473,11492014-fractured,4.00,871,"['311', '310', '197', '42', '11']",94.0,0,1
52474,11836711-anasazi,4.19,37,"['16', '14', '5', '2', '0']",95.0,0,1
52475,10815662-marked,3.70,6674,"['2109', '1868', '1660', '647', '390']",84.0,0,1
52476,11330278-wayward-son,3.85,238,"['77', '78', '59', '19', '5']",90.0,0,1


## Autor

| **Nombre Original (Hito 1)**                     | **Nombre Simplificado (para la implementación)** | **Tipo de Atributo**     |
|--------------------------------------------------|--------------------------------------------------|---------------------------|
| Nombre                                           | `nombre`                                         | `String`                  |
| Número de Trabajos                               | `num_trabajos`                                   | `Int`                     |
| Género                                           | `genero`                                         | `String`                  |
| Nacimiento                                       | `nacimiento`                                     | `String`                  |
| Muerte                                           | `muerte`                                         | `String`                  |
| Puntuación promedio                              | `puntuacion_prom`                            | `Float`                   |
| Número de votos                                   | `num_votos`                                      | `Int`                     |
| País                                             | `pais`                                           | `String`                  |


In [11]:
# Creamos DataFrame Autor:
Autor0 = unique_authors.copy()
Autor0.columns

# Mantener solo las columnas especificadas:
columns_to_keep = ['name_clean', 'workcount', 'gender', 'born', 'died', 'average_rate', 'rating_count', 'country']
Autor0 = Autor0[columns_to_keep]

# Renombramos los atributos a utilizar:
Autor0.rename(columns={
        'name_clean': 'nombre',
        'workcount': 'num_trabajos',
        'gender': 'genero',
        'born': 'nacimiento',
        'died': 'muerte',
        'average_rate': 'puntuacion_prom',
        'rating_count': 'num_votos',
        'country': 'pais'
    }, inplace=True)

# Resultado final
Autor0

,nombre,num_trabajos,genero,nacimiento,muerte,puntuacion_prom,num_votos,pais
24,lexy v. golden,30,unknown,NaN,NaN,4.05,514,NaN
38,جهاد عبد الحميد,1,unknown,NaN,NaN,3.52,591,NaN
51,keith roberts,118,male,1935-09-20,2000-10-05,3.69,5402,United Kingdom
112,sean fay wolfe,25,unknown,NaN,NaN,4.61,1737,NaN
127,karren brady,6,unknown,NaN,NaN,3.73,940,NaN
...,...,...,...,...,...,...,...,...
209508,susan fraser king,4,unknown,NaN,NaN,3.78,6060,NaN
209509,manda scott,21,female,NaN,NaN,4.07,11936,United Kingdom
209513,richard bandler,95,unknown,NaN,NaN,4.01,8053,NaN
209515,augusto cury,78,unknown,1958-10-02,NaN,3.82,10450,Macao


# 3. Limpieza, Creacion de Datos y de Tablas Finales

In [12]:
# Largos actuales de tablas:
print(len(Libro0), len(Valoracion0), len(Autor0))

52478 52478 17918


## 3.1 Libro

In [13]:
# Visualizacion de los datos
Libro0.head()

,bookid,titulo,autor,descripcion,idioma,generos,personajes,formato,paginas,editor,fecha_pub,fecha_pub_inicial,premios,ambientacion
0,2767052-the-hunger-games,The Hunger Games,suzanne collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",Hardcover,374,Scholastic Press,09/14/08,NaN,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Pane..."
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,j.k. rowling,There is a door at the end of a silent corrido...,English,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",Paperback,870,Scholastic Inc.,09/28/04,06/21/03,['Bram Stoker Award for Works for Young Reader...,['Hogwarts School of Witchcraft and Wizardry (...
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,harper lee,The unforgettable novel of a childhood in a sl...,English,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",Paperback,324,Harper Perennial Modern Classics,05/23/06,07/11/60,"['Pulitzer Prize for Fiction (1961)', 'Audie A...","['Maycomb, Alabama (United States)']"
3,1885.Pride_and_Prejudice,Pride and Prejudice,jane austen,Alternate cover edition of ISBN 9780679783268S...,English,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",Paperback,279,Modern Library,10/10/00,01/28/13,[],"['United Kingdom', 'Derbyshire, England (Unite..."
4,41865.Twilight,Twilight,stephenie meyer,About three things I was absolutely positive.\...,English,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",Paperback,501,"Little, Brown and Company",09/06/06,10/05/05,"['Georgia Peach Book Award (2007)', 'Buxtehude...","['Forks, Washington (United States)', 'Phoenix..."


In [14]:
# Contar valores nulos en cada columna del DataFrame Libro
null_counts = Libro0.isnull().sum()
print(null_counts)

bookid                   0
titulo                   0
autor                    0
descripcion           1338
idioma                3806
generos                  0
personajes               0
formato               1473
paginas               2347
editor                3696
fecha_pub              880
fecha_pub_inicial    21326
premios                  0
ambientacion             0
dtype: int64


In [15]:
# RELLENO CON DATOS FALSOS:

# Rellenar datos nulos en la columna 'descripcion' con descripciones falsas de 100 caracteres
Libro0['descripcion'] = Libro0['descripcion'].apply(lambda x: x if pd.notnull(x) else fake.text(max_nb_chars=100))

# Rellenar datos nulos en la columna 'idioma' con idiomas falsos
Libro0['idioma'] = Libro0['idioma'].apply(lambda x: x if pd.notnull(x) else fake.language_name())

# Rellenar datos nulos en la columna 'generos' con géneros falsos o cuando el dato es '[]':
Libro0['generos'] = Libro0['generos'].apply(lambda x: x if pd.notnull(x) and x != '[]' else fake.random_element(elements=('Fiction', 'Non-Fiction', 'Fantasy', 'Science Fiction', 'Mystery', 'Romance', 'Thriller', 'Horror', 'Biography', 'History', 'Self-Help', 'Cooking', 'Travel', 'Art', 'Science', 'Math', 'Poetry', 'Drama', 'Comics', 'Children', 'Young Adult', 'Adult')))

# Rellenar datos nulos en la columna 'personajes' con nombres de personajes falsos o cuando el dato es '[]':
Libro0['personajes'] = Libro0['personajes'].apply(lambda x: x if pd.notnull(x) and x != '[]' else fake.name())

# Rellenar datos nulos en la columna 'formato' con 'Hardcover' o 'Paperback'
Libro0['formato'] = Libro0['formato'].apply(lambda x: x if pd.notnull(x) else fake.random_element(elements=('Hardcover', 'Paperback')))

# Rellenar datos nulos en la columna 'paginas' con números aleatorios entre 50 y 1000
Libro0['paginas'] = Libro0['paginas'].apply(lambda x: x if pd.notnull(x) else fake.random_int(min=50, max=1000))

# Rellenar datos nulos en la columna 'editor' con nombres de editoriales falsas
Libro0['editor'] = Libro0['editor'].apply(lambda x: x if pd.notnull(x) else fake.company())

# Rellenar datos nulos en la columna 'fecha_pub' con fechas aleatorias entre 1900 y 2021 (en formato MM/DD/YY)
Libro0['fecha_pub'] = Libro0['fecha_pub'].apply(lambda x: x if pd.notnull(x) and not isinstance(x, str) else fake.date_between(start_date='-121y', end_date='today').strftime('%m/%d/%y'))

# Rellenar datos nulos en la columna 'fecha_pub_inicial' con fechas aleatorias entre 1900 y 2021 (en formato DD/MM/YY)
Libro0['fecha_pub_inicial'] = Libro0['fecha_pub_inicial'].apply(lambda x: x if pd.notnull(x) and not isinstance(x, str) else fake.date_between(start_date='-121y', end_date='today').strftime('%m/%d/%y'))

# Rellenar datos nulos en la columna 'premios' con nombres de premios falsos o cuando el dato es '[]':
Libro0['premios'] = Libro0['premios'].apply(lambda x: x if pd.notnull(x) and x != '[]' else fake.text(max_nb_chars=50))

# Rellenar datos nulos en la columna 'ambientacion' con nombres de lugares falsos o cuando el dato es '[]':
Libro0['ambientacion'] = Libro0['ambientacion'].apply(lambda x: x if pd.notnull(x) and x != '[]' else fake.city())


In [16]:
Libro0

,bookid,titulo,autor,descripcion,idioma,generos,personajes,formato,paginas,editor,fecha_pub,fecha_pub_inicial,premios,ambientacion
0,2767052-the-hunger-games,The Hunger Games,suzanne collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",Hardcover,374,Scholastic Press,10/08/12,10/17/90,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Pane..."
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,j.k. rowling,There is a door at the end of a silent corrido...,English,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",Paperback,870,Scholastic Inc.,10/29/27,04/28/57,['Bram Stoker Award for Works for Young Reader...,['Hogwarts School of Witchcraft and Wizardry (...
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,harper lee,The unforgettable novel of a childhood in a sl...,English,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",Paperback,324,Harper Perennial Modern Classics,09/16/61,04/28/27,"['Pulitzer Prize for Fiction (1961)', 'Audie A...","['Maycomb, Alabama (United States)']"
3,1885.Pride_and_Prejudice,Pride and Prejudice,jane austen,Alternate cover edition of ISBN 9780679783268S...,English,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",Paperback,279,Modern Library,05/08/39,06/05/65,Real worry two according.,"['United Kingdom', 'Derbyshire, England (Unite..."
4,41865.Twilight,Twilight,stephenie meyer,About three things I was absolutely positive.\...,English,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",Paperback,501,"Little, Brown and Company",08/10/22,06/15/74,"['Georgia Peach Book Award (2007)', 'Buxtehude...","['Forks, Washington (United States)', 'Phoenix..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52473,11492014-fractured,Fractured,cheri schmidt,The Fateful Trilogy continues with Fractured. ...,English,"['Vampires', 'Paranormal', 'Young Adult', 'Rom...",Anthony Charles,Nook,0,Cheri Schmidt,05/10/22,03/26/98,Almost eye degree action none.,Lake Rachel
52474,11836711-anasazi,Anasazi,emma michaels,"'Anasazi', sequel to 'The Thirteenth Chime' by...",English,"['Mystery', 'Young Adult']",Brandon Combs,Paperback,190,Bokheim Publishing,09/25/18,08/25/09,Window pass eight.,Lake Tyler
52475,10815662-marked,Marked,kim richardson,--READERS FAVORITE AWARDS WINNER 2011--Sixteen...,English,"['Fantasy', 'Young Adult', 'Paranormal', 'Ange...",Michael Taylor,Paperback,280,CreateSpace,03/21/96,08/06/26,"[""Readers' Favorite Book Award (2011)""]",Gonzaleztown
52476,11330278-wayward-son,Wayward Son,tom pollack,A POWERFUL TREMOR UNEARTHS AN ANCIENT SECRETBu...,English,"['Fiction', 'Mystery', 'Historical Fiction', '...",Cassie Lamb,Paperback,507,Cascada Productions,09/23/07,09/15/71,Entire focus condition television shake group.,Tonyburgh


In [17]:
# Contar valores nulos en cada columna del DataFrame Libro
null_counts = Libro0.isnull().sum()
print(null_counts)

bookid               0
titulo               0
autor                0
descripcion          0
idioma               0
generos              0
personajes           0
formato              0
paginas              0
editor               0
fecha_pub            0
fecha_pub_inicial    0
premios              0
ambientacion         0
dtype: int64


In [18]:
# Reemplazar valores no enteros en la columna 'paginas' con enteros aleatorios entre 50 y 1500
Libro0['paginas'] = Libro0['paginas'].apply(lambda x: x if isinstance(x, int) else fake.random_int(min=50, max=1500))

assert Libro0['paginas'].dtype == int

In [19]:
# Convertir la columna 'fecha_pub' a formato datetime donde los datos se encuentran en formato MM/DD/YY:
Libro0['fecha_pub'] = pd.to_datetime(Libro0['fecha_pub'], format='%m/%d/%y', errors='coerce')

# Extraer solo la parte de la fecha
Libro0['fecha_pub'] = Libro0['fecha_pub'].dt.date

assert Libro0['fecha_pub'].dtype == datetime

In [20]:
# Convertir la columna 'fecha_pub_inicial' a formato datetime donde los datos se encuentran en formato MM/DD/YY:
Libro0['fecha_pub_inicial'] = pd.to_datetime(Libro0['fecha_pub_inicial'], format='%m/%d/%y', errors='coerce')

# Extraer solo la parte de la fecha
Libro0['fecha_pub_inicial'] = Libro0['fecha_pub_inicial'].dt.date

assert Libro0['fecha_pub_inicial'].dtype == datetime

In [21]:
# UNICIDAD DE LA LLAVE BOOOKID EN LA TABLA LIBRO:

# Contar el número de duplicados que se eliminarán
num_duplicados = Libro0.duplicated(subset='bookid').sum()
print(f"Se eliminarán {num_duplicados} duplicados de la tabla Libro.")

# Eliminar filas duplicadas en la tabla Libro basadas en el bookid
Libro0 = Libro0.drop_duplicates(subset='bookid', keep='first')


# Verificar si hay duplicados
assert Libro0['bookid'].is_unique, 'Hay llaves bookid duplicadas en tabla Libro si falla el assert!'

Se eliminarán 54 duplicados de la tabla Libro.


In [22]:
# TABLA FINAL DE LIBRO:
Libro = Libro0.copy()
Libro

,bookid,titulo,autor,descripcion,idioma,generos,personajes,formato,paginas,editor,fecha_pub,fecha_pub_inicial,premios,ambientacion
0,2767052-the-hunger-games,The Hunger Games,suzanne collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",Hardcover,291,Scholastic Press,2012-10-08,1990-10-17,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Pane..."
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,j.k. rowling,There is a door at the end of a silent corrido...,English,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",Paperback,577,Scholastic Inc.,2027-10-29,2057-04-28,['Bram Stoker Award for Works for Young Reader...,['Hogwarts School of Witchcraft and Wizardry (...
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,harper lee,The unforgettable novel of a childhood in a sl...,English,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",Paperback,603,Harper Perennial Modern Classics,2061-09-16,2027-04-28,"['Pulitzer Prize for Fiction (1961)', 'Audie A...","['Maycomb, Alabama (United States)']"
3,1885.Pride_and_Prejudice,Pride and Prejudice,jane austen,Alternate cover edition of ISBN 9780679783268S...,English,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",Paperback,501,Modern Library,2039-05-08,2065-06-05,Real worry two according.,"['United Kingdom', 'Derbyshire, England (Unite..."
4,41865.Twilight,Twilight,stephenie meyer,About three things I was absolutely positive.\...,English,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",Paperback,169,"Little, Brown and Company",2022-08-10,1974-06-15,"['Georgia Peach Book Award (2007)', 'Buxtehude...","['Forks, Washington (United States)', 'Phoenix..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52473,11492014-fractured,Fractured,cheri schmidt,The Fateful Trilogy continues with Fractured. ...,English,"['Vampires', 'Paranormal', 'Young Adult', 'Rom...",Anthony Charles,Nook,1097,Cheri Schmidt,2022-05-10,1998-03-26,Almost eye degree action none.,Lake Rachel
52474,11836711-anasazi,Anasazi,emma michaels,"'Anasazi', sequel to 'The Thirteenth Chime' by...",English,"['Mystery', 'Young Adult']",Brandon Combs,Paperback,111,Bokheim Publishing,2018-09-25,2009-08-25,Window pass eight.,Lake Tyler
52475,10815662-marked,Marked,kim richardson,--READERS FAVORITE AWARDS WINNER 2011--Sixteen...,English,"['Fantasy', 'Young Adult', 'Paranormal', 'Ange...",Michael Taylor,Paperback,963,CreateSpace,1996-03-21,2026-08-06,"[""Readers' Favorite Book Award (2011)""]",Gonzaleztown
52476,11330278-wayward-son,Wayward Son,tom pollack,A POWERFUL TREMOR UNEARTHS AN ANCIENT SECRETBu...,English,"['Fiction', 'Mystery', 'Historical Fiction', '...",Cassie Lamb,Paperback,1030,Cascada Productions,2007-09-23,1971-09-15,Entire focus condition television shake group.,Tonyburgh


## 3.2 Valoracion

In [23]:
# Visualizacion de los datos
Valoracion0.head()

,bookid,calificacion,num_calificaciones,estrellas,porcentaje_likes,puntuacion_bbe,votos_bbe
0,2767052-the-hunger-games,4.33,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,2993816,30516
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,4.50,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,2632233,26923
2,2657.To_Kill_a_Mockingbird,4.28,4501075,"['2363896', '1333153', '573280', '149952', '80...",95.0,2269402,23328
3,1885.Pride_and_Prejudice,4.26,2998241,"['1617567', '816659', '373311', '113934', '767...",94.0,1983116,20452
4,41865.Twilight,3.60,4964519,"['1751460', '1113682', '1008686', '542017', '5...",78.0,1459448,14874


In [24]:
# Contar valores nulos en cada columna del DataFrame Valoracion:
null_counts = Valoracion0.isnull().sum()
print(null_counts)

bookid                  0
calificacion            0
num_calificaciones      0
estrellas               0
porcentaje_likes      622
puntuacion_bbe          0
votos_bbe               0
dtype: int64


In [25]:
# RELLENO CON DATOS FALSOS:

# Rellenar datos nulos de porcentaje de likes con valores aleatorios entre 0 y 100:
Valoracion0['porcentaje_likes'] = Valoracion0['porcentaje_likes'].apply(lambda x: x if pd.notnull(x) else fake.random_int(min=0, max=100))

# Transformamos a Int los datos de la columna 'porcentaje_likes':
Valoracion0['porcentaje_likes'] = Valoracion0['porcentaje_likes'].astype(int)

In [26]:
# Contar valores nulos en cada columna del DataFrame Valoracion:
null_counts = Valoracion0.isnull().sum()
print(null_counts)

bookid                0
calificacion          0
num_calificaciones    0
estrellas             0
porcentaje_likes      0
puntuacion_bbe        0
votos_bbe             0
dtype: int64


In [27]:
# UNICIDAD DE LA LLAVE BOOKID EN VALORACION:

# Contar el número de duplicados que se eliminarán
num_duplicados = Valoracion0.duplicated(subset='bookid').sum()
print(f"Se eliminarán {num_duplicados} duplicados de la tabla Valoracion.")

# Eliminar filas duplicadas en la tabla Libro basadas en el bookid
Valoracion0 = Valoracion0.drop_duplicates(subset='bookid', keep='first')

# Verificar si hay duplicados
assert Valoracion0['bookid'].is_unique, 'Hay llaves bookid duplicadas en tabla Valoracion si falla el assert!'

Se eliminarán 54 duplicados de la tabla Valoracion.


In [28]:
# TABLA FINAL DE VALORACION:
Valoracion = Valoracion0.copy()
Valoracion.head()

,bookid,calificacion,num_calificaciones,estrellas,porcentaje_likes,puntuacion_bbe,votos_bbe
0,2767052-the-hunger-games,4.33,6376780,"['3444695', '1921313', '745221', '171994', '93...",96,2993816,30516
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,4.50,2507623,"['1593642', '637516', '222366', '39573', '14526']",98,2632233,26923
2,2657.To_Kill_a_Mockingbird,4.28,4501075,"['2363896', '1333153', '573280', '149952', '80...",95,2269402,23328
3,1885.Pride_and_Prejudice,4.26,2998241,"['1617567', '816659', '373311', '113934', '767...",94,1983116,20452
4,41865.Twilight,3.60,4964519,"['1751460', '1113682', '1008686', '542017', '5...",78,1459448,14874


## 3.3 Autor

In [29]:
# Visualizacion de los datos
Autor0.head()

,nombre,num_trabajos,genero,nacimiento,muerte,puntuacion_prom,num_votos,pais
24,lexy v. golden,30,unknown,NaN,NaN,4.05,514,NaN
38,جهاد عبد الحميد,1,unknown,NaN,NaN,3.52,591,NaN
51,keith roberts,118,male,1935-09-20,2000-10-05,3.69,5402,United Kingdom
112,sean fay wolfe,25,unknown,NaN,NaN,4.61,1737,NaN
127,karren brady,6,unknown,NaN,NaN,3.73,940,NaN


In [30]:
# Contar valores nulos en cada columna del DataFrame Autor:
null_counts = Autor0.isnull().sum()
print(null_counts)

nombre                 0
num_trabajos           0
genero                 0
nacimiento         10671
muerte             14589
puntuacion_prom        0
num_votos              0
pais                8899
dtype: int64


In [31]:
# RELLENO CON DATOS FALSOS:

# Rellenar datos nulos en la columna 'genero' con 'male' o 'female' si el genero es 'unknown'
Autor0['genero'] = Autor0['genero'].apply(lambda x: x if x != 'unknown' else fake.random_element(elements=('male', 'female')))

# Rellenar datos nulos en la columna 'nacimiento' con fechas aleatorias entre 1800 y 2002 (en formato YYYY-MM-DD)
Autor0['nacimiento'] = Autor0['nacimiento'].apply(lambda x: x if pd.notnull(x) else fake.date_of_birth(minimum_age=18, maximum_age=100).strftime('%Y-%m-%d'))

# Convertir la columna 'nacimiento' a tipo datetime:
Autor0['nacimiento'] = pd.to_datetime(Autor0['nacimiento'])

# Rellenar datos nulos en la columna 'muerte' con fechas aleatorias entre 1900 y 2021 (en formato YYYY-MM-DD) tal que la persona tenga almenos 18 años antes de morir:
Autor0['muerte'] = Autor0.apply(lambda row: row['muerte'] if pd.notnull(row['muerte']) else fake.date_between(start_date=row['nacimiento'] + pd.DateOffset(years=18), end_date='today').strftime('%Y-%m-%d'), axis=1)

# Convertir la columna 'muerte' a tipo datetime:
Autor0['muerte'] = pd.to_datetime(Autor0['muerte'])

# Rellenar la columna 'pais' con nombres de paises falsos:
Autor0['pais'] = Autor0['pais'].apply(lambda x: x if pd.notnull(x) else fake.country())

In [32]:
# Contar valores nulos en cada columna del DataFrame Autor:
null_counts = Autor0.isnull().sum()
print(null_counts)

nombre             0
num_trabajos       0
genero             0
nacimiento         0
muerte             0
puntuacion_prom    0
num_votos          0
pais               0
dtype: int64


In [33]:
# Ajusta el indice de Autor0 para que parta en 0:
Autor0.reset_index(drop=True, inplace=True)

In [34]:
# TABLA FINAL DE AUTOR:
Autor = Autor0.copy()
Autor.head()

,nombre,num_trabajos,genero,nacimiento,muerte,puntuacion_prom,num_votos,pais
0,lexy v. golden,30,female,1938-10-11,2013-02-14,4.05,514,Montenegro
1,جهاد عبد الحميد,1,male,1934-05-06,1982-04-01,3.52,591,Eritrea
2,keith roberts,118,male,1935-09-20,2000-10-05,3.69,5402,United Kingdom
3,sean fay wolfe,25,male,1958-08-23,2017-10-26,4.61,1737,Hungary
4,karren brady,6,female,1930-08-05,1970-05-16,3.73,940,Hong Kong


# 4. Recorte Final de Filas (eliminación de casos problemáticos)

In [35]:
# Largos actuales de tablas:
print(len(Libro), len(Valoracion), len(Autor))

52424 52424 17918


In [36]:
# Verifica que todos los autores en la tabla Libro estén en la tabla Autor (para efectividad de la llave foránea)
autores_faltantes = Libro[~Libro['autor'].isin(Autor['nombre'])]['autor']

# Imprimir los autores que faltan
if not autores_faltantes.empty:
    print(f"{len(autores_faltantes)} no se encuentran en la tabla Autor:")
    print(autores_faltantes.tolist())
else:
    print("Todos los autores en la tabla Libro están presentes en la tabla Autor.")

6982 no se encuentran en la tabla Autor:
['richard adams', 'arthur miller', 'thomas harris', 'سید مرتضی مصطفوی', 'arthur miller', 'kyle keyes', 'diana forbes', 'abraham verghese', 'diamante lavendar', 'kyle keyes', 'kyle keyes', 'suzanne boisvert', 'سید مرتضی مصطفوی', 'hubert seljr.', 'aimee cabo nikolov', 'c. toni graham', 'traci medford-rosow', 'kenneth thomas', 'sergio cobo', 'peter b. forster', 'kathleen zamboni mccormick', 'james mcbride', 'vera jane cook', 'thomas harris', 'kenneth jarrett singleton', 'ammar habib', 'karen glista', 'andrea luhman', 'trevor alan foris', 'm. robinson', 'maria nhambu', 'fred misurella', 'anne willow', 'kenneth thomas', 'kyle keyes', 'vincent panettiere', 'mary stewart', 'd.s. smith', 'm. robinson', 'omar farhad', 'afroz alam', 'michael g. kramer', 'michel guyon', 'yuehai xiao', 'ammar habib', 'rohith s. katbamna', 'simon okill', 'karen prince', 'c. toni graham', 'gillian dance', 'rebecca donovan', 'gabriel f.w. koch', 'سید مرتضی مصطفوی', 'maria nham

In [37]:
# Identificar los IDs de los libros asociados a los autores faltantes para eliminarlos de la tabla Libro

libros_faltantes_ids = Libro[Libro['autor'].isin(autores_faltantes)]['bookid']

assert len(libros_faltantes_ids) == len(autores_faltantes)

In [38]:
# Eliminar las filas de la tabla Valoracion cuyos bookid están en libros_faltantes_ids
Valoracion = Valoracion[~Valoracion['bookid'].isin(libros_faltantes_ids)]

# Eliminar los libros de la tabla Libro cuyos autores están en autores_faltantes
Libro = Libro[~Libro['autor'].isin(autores_faltantes)]

assert len(Valoracion) == len(Libro)

In [39]:
duplicados = Autor[Autor.duplicated(subset=['nombre'], keep=False)] #hay 4 nombres repetidos. Se eliminaran las 8 filas duplicadas. al ejecutarlo denuevo tras un runall deberia dar vacio.
duplicados

,nombre,num_trabajos,genero,nacimiento,muerte,puntuacion_prom,num_votos,pais
4979,hesiod,102,female,1958-11-14,1983-02-22,3.78,19039,Greece
5826,miroslav antić,2,male,2003-10-29,2024-02-17,4.66,104,Syrian Arab Republic
5827,miroslav antić,25,male,1932-03-14,1986-06-24,4.59,2085,Serbia
6923,hesiod,1,male,1931-08-28,1955-06-26,3.47,180,Zimbabwe
7131,mike mignola,1,female,1931-03-27,1968-05-08,3.78,469,San Marino
11451,joshua e. hanft,9,female,1979-11-23,2011-06-01,3.88,390,Vietnam
13271,mike mignola,1011,male,1960-09-16,2004-06-08,4.11,321613,United States
17287,joshua e. hanft,1,male,2006-01-07,2024-06-28,4.14,950,Dominican Republic


In [40]:
# ELIMINAR LLAVES DUPLICADAS EN Autor:

# Eliminar duplicados de la tabla Autor, eliminando las dos filas duplicadas:
Autor = Autor.drop(duplicados.index)

# Eliminar las filas en la tabla Libro cuyos autores están en la lista de duplicados
Libro = Libro[~Libro['autor'].isin(duplicados['nombre'])]

# Eliminar las filas en la tabla Valoracion cuyos bookid están en la tabla Libro
Valoracion = Valoracion[Valoracion['bookid'].isin(Libro['bookid'])]

# Verificar los resultados
assert len(Libro) == len(Valoracion)
assert len(Autor) == 17910 # (17918 - 8)

print(len(Libro), len(Valoracion), len(Autor))

45429 45429 17910


In [41]:
# VERIFICACION FINAL DE VALIDEZ DE LA LLAVE:

# Verifica que todos los autores en la tabla Libro estén en la tabla Autor
autores_faltantes = Libro[~Libro['autor'].isin(Autor['nombre'])]['autor']

# Imprimir los autores que faltan
if not autores_faltantes.empty:
    print(f"{len(autores_faltantes)} no se encuentran en la tabla Autor:")
    print(autores_faltantes.tolist())
else:
    print("Todos los autores en la tabla Libro están presentes en la tabla Autor. GG!")

Todos los autores en la tabla Libro están presentes en la tabla Autor. GG!


# 5. Visualizacion de Tablas Finales (Libro, Valoracion y Autor)

In [42]:
Libro.head()

,bookid,titulo,autor,descripcion,idioma,generos,personajes,formato,paginas,editor,fecha_pub,fecha_pub_inicial,premios,ambientacion
0,2767052-the-hunger-games,The Hunger Games,suzanne collins,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",Hardcover,291,Scholastic Press,2012-10-08,1990-10-17,['Locus Award Nominee for Best Young Adult Boo...,"['District 12, Panem', 'Capitol, Panem', 'Pane..."
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,j.k. rowling,There is a door at the end of a silent corrido...,English,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",Paperback,577,Scholastic Inc.,2027-10-29,2057-04-28,['Bram Stoker Award for Works for Young Reader...,['Hogwarts School of Witchcraft and Wizardry (...
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,harper lee,The unforgettable novel of a childhood in a sl...,English,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",Paperback,603,Harper Perennial Modern Classics,2061-09-16,2027-04-28,"['Pulitzer Prize for Fiction (1961)', 'Audie A...","['Maycomb, Alabama (United States)']"
3,1885.Pride_and_Prejudice,Pride and Prejudice,jane austen,Alternate cover edition of ISBN 9780679783268S...,English,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",Paperback,501,Modern Library,2039-05-08,2065-06-05,Real worry two according.,"['United Kingdom', 'Derbyshire, England (Unite..."
4,41865.Twilight,Twilight,stephenie meyer,About three things I was absolutely positive.\...,English,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",Paperback,169,"Little, Brown and Company",2022-08-10,1974-06-15,"['Georgia Peach Book Award (2007)', 'Buxtehude...","['Forks, Washington (United States)', 'Phoenix..."


In [43]:
Valoracion.head()

,bookid,calificacion,num_calificaciones,estrellas,porcentaje_likes,puntuacion_bbe,votos_bbe
0,2767052-the-hunger-games,4.33,6376780,"['3444695', '1921313', '745221', '171994', '93...",96,2993816,30516
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,4.50,2507623,"['1593642', '637516', '222366', '39573', '14526']",98,2632233,26923
2,2657.To_Kill_a_Mockingbird,4.28,4501075,"['2363896', '1333153', '573280', '149952', '80...",95,2269402,23328
3,1885.Pride_and_Prejudice,4.26,2998241,"['1617567', '816659', '373311', '113934', '767...",94,1983116,20452
4,41865.Twilight,3.60,4964519,"['1751460', '1113682', '1008686', '542017', '5...",78,1459448,14874


In [44]:
Autor.head()

,nombre,num_trabajos,genero,nacimiento,muerte,puntuacion_prom,num_votos,pais
0,lexy v. golden,30,female,1938-10-11,2013-02-14,4.05,514,Montenegro
1,جهاد عبد الحميد,1,male,1934-05-06,1982-04-01,3.52,591,Eritrea
2,keith roberts,118,male,1935-09-20,2000-10-05,3.69,5402,United Kingdom
3,sean fay wolfe,25,male,1958-08-23,2017-10-26,4.61,1737,Hungary
4,karren brady,6,female,1930-08-05,1970-05-16,3.73,940,Hong Kong


In [45]:
print(f"Largo de la tabla Libro: {len(Libro)}")
print(f"Largo de la tabla Valoracion: {len(Valoracion)}")
print(f"Largo de la tabla Autor: {len(Autor)}")

Largo de la tabla Libro: 45429
Largo de la tabla Valoracion: 45429
Largo de la tabla Autor: 17910


### Largos Finales de las Tablas:

| **Tabla**     | **Largo**  |
|---------------|------------|
| Libro         | 45,429     |
| Valoracion    | 45,429     |
| Autor         | 17,910     |


In [46]:
# Prueba Final:
# Verificar unicidad de bookid en la tabla Valoracion
assert Valoracion['bookid'].is_unique, 'Hay llaves bookid duplicadas en la tabla Valoracion'

# Verificar unicidad de bookid en la tabla Libro
assert Libro['bookid'].is_unique, 'Hay llaves bookid duplicadas en la tabla Libro'

# Verificar unicidad de nombre en la tabla Autor
assert Autor['nombre'].is_unique, 'Hay llaves nombre duplicadas en la tabla Autor'

print("Todas las llaves son únicas en las tablas Valoracion, Libro y Autor.")

Todas las llaves son únicas en las tablas Valoracion, Libro y Autor.


# 6. Exportar Datos (.csv)

In [47]:
# Cambiar [] por {} solo en el inicio y final de los arreglos en Libro y Valoracion:
Libro['generos'] = Libro['generos'].str.replace(r'^\[', '{').str.replace(r'\]$', '}')
Libro['personajes'] = Libro['personajes'].str.replace(r'^\[', '{').str.replace(r'\]$', '}')
Libro['premios'] = Libro['premios'].str.replace(r'^\[', '{').str.replace(r'\]$', '}')
Libro['ambientacion'] = Libro['ambientacion'].str.replace(r'^\[', '{').str.replace(r'\]$', '}')
Valoracion['estrellas'] = Valoracion['estrellas'].str.replace(r'^\[', '{').str.replace(r'\]$', '}')

# Elimina los {} que no estan en los extremos de las columnas:
# Definir la función de limpieza
def limpiar_premios(premios):
    return re.sub(r'(?<!^)\{(.*?)\}(?!$)', r'\1', premios)

# Aplicar la función a la columna 'premios' del DataFrame 'Libro'
Libro['generos'] = Libro['generos'].apply(limpiar_premios)
Libro['personajes'] = Libro['personajes'].apply(limpiar_premios)
Libro['premios'] = Libro['premios'].apply(limpiar_premios)
Libro['ambientacion'] = Libro['ambientacion'].apply(limpiar_premios)
Valoracion['estrellas'] = Valoracion['estrellas'].apply(limpiar_premios)

# Añadir {} si corresponde
Libro['generos'] = Libro['generos'].apply(lambda x: '{' + x + '}' if not x.startswith('{') else x)
Libro['personajes'] = Libro['personajes'].apply(lambda x: '{' + x + '}' if not x.startswith('{') else x)
Libro['premios'] = Libro['premios'].apply(lambda x: '{' + x + '}' if not x.startswith('{') else x)
Libro['ambientacion'] = Libro['ambientacion'].apply(lambda x: '{' + x + '}' if not x.startswith('{') else x)
Valoracion['estrellas'] = Valoracion['estrellas'].apply(lambda x: '{' + x + '}' if not x.startswith('{') else x)

# Elimina todas las comillas simples de las cuatro columnas anteriores:
Libro['generos'] = Libro['generos'].str.replace("'", "")
Libro['personajes'] = Libro['personajes'].str.replace("'", "")
Libro['premios'] = Libro['premios'].str.replace("'", "")
Libro['ambientacion'] = Libro['ambientacion'].str.replace("'", "")
Valoracion['estrellas'] = Valoracion['estrellas'].str.replace("'", "")

# Eliminas las comas ',' que no tienen texto en su lado derecho:
Libro['generos'] = Libro['generos'].str.replace(',}', '}')
Libro['personajes'] = Libro['personajes'].str.replace(',}', '}')
Libro['premios'] = Libro['premios'].str.replace(',}', '}')
Libro['ambientacion'] = Libro['ambientacion'].str.replace(',}', '}')
Valoracion['estrellas'] = Valoracion['estrellas'].str.replace(',}', '}')

# Reemplaza las comas ',,' por ',' en las cuatro columnas anteriores:
Libro['generos'] = Libro['generos'].str.replace(',,', ',')
Libro['personajes'] = Libro['personajes'].str.replace(',,', ',')
Libro['premios'] = Libro['premios'].str.replace(',,', ',')
Libro['ambientacion'] = Libro['ambientacion'].str.replace(',,', ',')

In [48]:
# Buscar 'French-American' en algún elemento de algún array de la columna 'premios'
french_american_books = Libro[Libro['premios'].apply(lambda x: 'French-American' in x)]

# Mostrar los resultados
french_american_books['premios']

538      {[French-American Foundation Translation Prize...
682      {[BTBA Best Translated Book Award Nominee for ...
1529     {[Magnesia Litera for Translation (Litera za p...
2814     {[Prix Goncourt (2006), Independent Foreign Fi...
3111     {[Prix Médicis (1978), Mekka-prijs (1996), Fre...
3142     {[French-American Foundation Translation Prize...
7634     {[French-American Foundation Translation Prize...
10752    {[BTBA Best Translated Book Award Nominee for ...
11061    {[French-American Foundation Translation Prize...
12035    {[Grand Prix RTL-Lire (2014), Wellcome Book Pr...
28808    {[Prix Femina Vacaresco (1994), French-America...
28970    {[Prix Goncourt des Lycéens (2007), Independen...
33061    {[French-American Foundation Translation Prize...
39723    {[Prix Renaudot (2000), Prix Goncourt des Lycé...
45163    {[Prix Goncourt (1978), French-American Founda...
47567    {[French-American Foundation Translation Prize...
51150    {[BTBA Best Translated Book Award Nominee for .

In [54]:
# Buscar libros de George R.R. Martin en la columna 'autor'
george_rr_martin_books = Libro[Libro['autor'].str.contains("george r.r. martin", case=False, na=False)]

# Mostrar los resultados
george_rr_martin_books

,bookid,titulo,autor,descripcion,idioma,generos,personajes,formato,paginas,editor,fecha_pub,fecha_pub_inicial,premios,ambientacion
46,13496.A_Game_of_Thrones,A Game of Thrones,george r.r. martin,Here is the first volume in George R. R. Marti...,English,"{[Fantasy, Fiction, Epic Fantasy, Adult, Scien...","{[Brandon Stark, Catelyn Stark, Tyrion Lannist...",Mass Market Paperback,897,Bantam,2051-02-17,2041-01-10,"{[Nebula Award Nominee for Best Novel (1997), ...",{[The Lands of Ice and Fire]}
194,62291.A_Storm_of_Swords,A Storm of Swords,george r.r. martin,An alternate cover for this isbn can be found ...,English,"{[Fantasy, Fiction, Epic Fantasy, Science Fict...","{[Brandon Stark, Catelyn Stark, Tyrion Lannist...",Mass Market Paperback,834,Bantam,2049-07-14,2021-06-19,"{[Hugo Award Nominee for Best Novel (2001), Ne...","{[Westeros, Essos]}"
248,10572.A_Clash_of_Kings,A Clash of Kings,george r.r. martin,A comet the color of blood and flame cuts acro...,English,"{[Fantasy, Fiction, Epic Fantasy, Science Fict...","{[Brandon Stark, Catelyn Stark, Tyrion Lannist...",Paperback,593,Bantam,1992-12-28,1987-09-26,"{[Nebula Award Nominee for Best Novel (1999), ...","{[The Lands of Ice and Fire, 'Kings Landing', ..."
429,13497.A_Feast_for_Crows,A Feast for Crows,george r.r. martin,"Crows will fight over a dead man's flesh, and ...",English,"{[Fantasy, Fiction, Epic Fantasy, Science Fict...","{[Arya Stark, Jaime Lannister, Petyr Baelish, ...",Mass Market Paperback,988,Bantam Books,2022-04-23,2003-06-06,"{[Hugo Award Nominee for Best Novel (2006), Lo...",{New Timothyville}
471,10664113-a-dance-with-dragons,A Dance with Dragons,george r.r. martin,Alternate cover edition of ASIN B004XISI4AIn t...,English,"{[Fantasy, Fiction, Epic Fantasy, Science Fict...","{[Brandon Stark, Tyrion Lannister, Daenerys Ta...",Kindle Edition,1485,Bantam,2004-11-08,2025-09-25,"{[Hugo Award Nominee for Best Novel (2012), Lo...",{East Jacobhaven}
1259,147915.A_Storm_of_Swords,A Storm of Swords: Blood and Gold,george r.r. martin,The Starks are scattered.Robb Stark may be Kin...,English,"{[Fantasy, Fiction, Epic Fantasy, High Fantasy...",{Chase Harris},Paperback,501,HarperVoyager,2028-10-30,1982-07-02,{[Tähtifantasia Award Nominee (2007)]},{South Misty}
1578,768889.A_Storm_of_Swords,A Storm of Swords: Steel and Snow,george r.r. martin,Winter approaches Westeros like an angry beast...,English,"{[Fantasy, Fiction, Epic Fantasy, High Fantasy...",{Jonathan Miller},Paperback,1095,HarperVoyager,2010-02-26,2014-06-21,{Get present talk great.},{North Rebecca}
1679,12177850-a-song-of-ice-and-fire,A Song of Ice and Fire,george r.r. martin,"For the first time, all five novels in the epi...",English,"{[Fantasy, Fiction, Science Fiction Fantasy, A...","{[Tyrion Lannister, Arya Stark, Khal Drogo, Ae...",Mass Market Paperback,477,Zatpix,1993-01-31,1975-07-21,{Stock big world catch team deal.},{Connieberg}
2899,9814682-a-song-of-ice-and-fire,A Song of Ice and Fire,george r.r. martin,George R. R. Martin's A Song of Ice and Fire s...,English,"{[Fantasy, Fiction, Epic Fantasy, Adventure, S...",{Victoria Martin},Mass Market Paperback,1055,Bantam,2011-05-15,2054-07-20,{Company foot tax few pressure ability base me...,{Burtonstad}
4322,39943621-fire-blood,Fire & Blood,george r.r. martin,With all the fire and fury fans have come to e...,English,"{[Fantasy, Fiction, Audiobook, Epic Fantasy, D...",{Brandi Thornton},Hardcover,1355,Bantam,1985-12-09,1995-10-06,"{[Locus Award Nominee for Collection (2019), G...",{East Emilymouth}


In [50]:
Libro['personajes']
# Buscar "Scarlett O'Hara" en la columna 'personajes' de Libro
scarlett_ohara_books = Libro[Libro['personajes'].str.contains("Scarlett O'Hara", case=False, na=False)]

# Mostrar los resultados
scarlett_ohara_books

# Cambiar comillas dobles por comillas simples en los arreglos de la columna 'generos'
Libro['generos'] = Libro['generos'].str.replace('"', "'")

# Cambiar comillas dobles por comillas simples en los arreglos de la columna 'personajes'
Libro['personajes'] = Libro['personajes'].str.replace('"', "'")

# Cambiar comillas dobles por comillas simples en los arreglos de la columna 'premios'
Libro['premios'] = Libro['premios'].str.replace('"', "'")

# Cambiar comillas dobles por comillas simples en los arreglos de la columna 'ambientacion'
Libro['ambientacion'] = Libro['ambientacion'].str.replace('"', "'")

# Cambiar comillas dobles por comillas simples en los arreglos de la columna 'estrellas'
Valoracion['estrellas'] = Valoracion['estrellas'].str.replace('[', '{').str.replace(']', '}')

`Descomentar` para guardar las 3 tablas en formato `.csv`, en particular, recordar que cada ejecución completa del código con `runall` genera algunos datos nuevos por los creados a mano.

In [51]:
# Exportar la tabla Libro a CSV
Libro.to_csv('Libro.csv', index=False)

# Exportar la tabla Valoracion a CSV
Valoracion.to_csv('Valoracion.csv', index=False)

# Exportar la tabla Autor a CSV
Autor.to_csv('Autor.csv', index=False)

In [55]:
from collections import Counter

# Unir todos los premios en una sola lista
all_premios = Libro['premios'].str.cat(sep=',').replace('{', '').replace('}', '').split(',')

# Contar la frecuencia de cada premio
premio_counts = Counter(all_premios)

# Encontrar el premio con mayor frecuencia
most_common_premio = premio_counts.most_common(1)
print(f"El premio que aparece con mayor frecuencia es: {most_common_premio[0][0]} con {most_common_premio[0][1]} apariciones.")

El premio que aparece con mayor frecuencia es:  LovelyBooks Leserpreis Nominee for Allgemeine Literatur (2009)] con 27 apariciones.


In [56]:
from collections import Counter

# Unir todos los géneros en una sola lista
all_generos = Libro['generos'].str.cat(sep=',').replace('{', '').replace('}', '').split(',')

# Contar la frecuencia de cada género
genero_counts = Counter(all_generos)

# Encontrar los géneros más comunes
most_common_generos = genero_counts.most_common(10)
print("Los géneros más comunes son:")
for genero, count in most_common_generos:
    print(f"{genero.strip()}: {count} apariciones")

Los géneros más comunes son:
Fiction: 22955 apariciones
Romance: 10993 apariciones
Contemporary: 8868 apariciones
Fantasy: 8603 apariciones
Young Adult: 7738 apariciones
Adult: 6385 apariciones
Novels: 5771 apariciones
Audiobook: 5715 apariciones
[Fiction: 5639 apariciones
Historical: 5548 apariciones
